<h1 align="center">Exploring Sacramento, California, with MongoDB</h1> 
<h3 align="center">Author: Stanislav Sajin</h3>


## General Summary
This document aims to wrangle OpenStreetMap data for the city of Sacramento, California. I chose this region because I currently reside here. This is not the report used for the project. For the report, please reffer to the file Report.pdf.

    - Parse the data for tag types and tag counts
    - Examine the validity of the data by looking at the tag keys within the XML file
    - Audit the data, which will involve the following:
        * Correct Street Name Typos (e.g. Placez to Plaza)
        * Removal of data with invalid keys, such as a nonsensical street names (e.g., 9311, 9322).
        * Normalization of abreviations (e.g., "St.", "St", "str", "Street" are changed into "Street")
        * Normalization of the postal code;
          -The zip needs to be between 94000-96000 (which corresponds more closely to Sacramento region and areas around it)
          -The zip can either be a 5 digit code or a five digit code folloed by 4 digits (e.g., 95832-1447)
          -The zip should not have non-digit strings.
    -Problems encountered
    -Exploring data with MongoDB
    





### Area Image

<div style="text-align:center"><img src ="Sacramento.jpg" /></div>

### Libraries

In [1]:
##Import all the libraries that will be used in this document
import pymongo
import os

### Data Source Information
I downloaded the data from https://mapzen.com/data/metro-extracts/. 

The OSM XML file can be downloaded here: https://s3.amazonaws.com/metro-extracts.mapzen.com/sacramento_california.osm.bz2

### Data File Size Information
I will be using getsize from os to measure the size of the uncompressed XML file. Since the file size given to me will be in bytes, I will divide the output by 1,000,000 to get the file size in MB

In [118]:
sizeXML=os.path.getsize('sacramento_california.osm')/1000000
sizeJSON=os.path.getsize('sacramento_california.osm.json')/1000000
print "The XML file size is "+str(sizeXML)+" MB"
print "The JSON file size is "+str(sizeJSON)+" MB"

The XML file size is 231 MB
The JSON file size is 459 MB


### Parse the data for tag types and tag counts
The following code chunk will run the code inside mapparser.py and will return a dictionary with the type of tags found in the XML file and the counts for each tag.

In [3]:
%run mapparser.py

defaultdict(<type 'int'>, {'node': 1028855, 'nd': 1180056, 'bounds': 1, 'member': 5565, 'tag': 757656, 'relation': 831, 'way': 100410, 'osm': 1})


In the code below I will run the tags.py and check the value of the keys in each tag to check for invalid keys. 

In [5]:
%run tags.py

<Element 'tag' at 0xa3ec860>
Segment #
<Element 'tag' at 0xa3ec128>
Elevation, end
<Element 'tag' at 0xa3ec400>
Elevation, start
<Element 'tag' at 0xa3ecc50>
Elevation, maximum
<Element 'tag' at 0xa3ec748>
Elevation, minimum
<Element 'tag' at 0xa3fd128>
Segment #
<Element 'tag' at 0xa3fd160>
Elevation, end
<Element 'tag' at 0xa44f0b8>
Elevation, start
<Element 'tag' at 0xa44f128>
Elevation, maximum
<Element 'tag' at 0xa44f198>
Elevation, minimum
<Element 'tag' at 0xa3f2518>
Segment #
<Element 'tag' at 0xa3f22b0>
Elevation, end
<Element 'tag' at 0xa3f2cc0>
Elevation, start
<Element 'tag' at 0xa3f2a58>
Elevation, maximum
<Element 'tag' at 0xa3f2390>
Elevation, minimum
<Element 'tag' at 0x3be99d68>
gost_7.67-2003
{'lower': 305683, 'lower_colon': 442468, 'other': 9489, 'problemchars': 16}


We have several keys with problem characters (16) that will be removed later when creating the structure of the json file. Next, I will lok at how many users we have contributed to our data.

In [7]:
%run users.py

set(['100222',
     '1007528',
     '101372',
     '101433',
     '1015635',
     '1016290',
     '102870',
     '102899',
     '103253',
     '103273',
     '10353',
     '103574',
     '104519',
     '104962',
     '1051550',
     '105462',
     '1058666',
     '1067273',
     '107257',
     '107566',
     '10786',
     '108149',
     '1087647',
     '108775',
     '109362',
     '109808',
     '1098648',
     '11006',
     '110126',
     '110263',
     '1102790',
     '11121',
     '11126',
     '11154',
     '112254',
     '1125908',
     '113518',
     '1136962',
     '1149057',
     '115064',
     '1151679',
     '1151882',
     '11547',
     '1155756',
     '115592',
     '115918',
     '1161559',
     '1164',
     '1168707',
     '1168909',
     '1169950',
     '1179821',
     '118021',
     '1195028',
     '1195114',
     '1196285',
     '1198074',
     '1198360',
     '119881',
     '120153',
     '1204',
     '1208453',
     '121241',
     '1214027',
     '1215176',
     '12

The next python script aims to run the audit and the data. The script will clean up the street names a bit and also remove an incorrect zipcode. 

In [19]:
%run audit.py

defaultdict(<type 'int'>, {'96816': 5, '95819-6138': 1, 'CA 95747': 2, 'CA 95834': 2, 'CA 95833': 1, 'CA 95832': 1, 'CA 95626': 1, '95609-0579': 1, 'CA 95819': 1, 'CA 95628': 1, 'CA 95602': 1, '96516': 2, 'CA 95648': 1, '95826-2625': 1, '95819-6024': 1, 'CA 95683': 1, '85834': 1, 'CA 95829': 1, 'CA 95826': 1, 'CA 95827': 1, 'CA 95822': 1, '95832-1447': 2, '98584': 1, 'CA 95616': 2, '95616-5270': 3, 'CA 95655': 1, u'CA 95650 \u200e': 1, 'CA 95659': 1})
{'1': set(['California Highway 16, House No. 1']),
 '102': set(['County Road 102']),
 '110,': set(['Promenade Circle #110,']),
 '18C': set(['County Road 18C']),
 '193': set(['Highway 193', 'Sw Cnr Sierra College Blvd / Sr 193']),
 '310': set(['Greenback Lane, Suite 310']),
 '5': set(['2nd St/Rte 5']),
 '50': set(['3rd St / Rte 50']),
 '5259': set(['PO Box 5259']),
 '9311': set(['9311']),
 '9321': set(['9321']),
 '95628-7416': set(['Fair Oaks Boulevard, Fair Oaks, CA 95628-7416']),
 '95683': set(['Pera Dr at Escuela Drive, Rancho Murieta, 

Problems Encountered:
- One of the issues with this dataset is that the street names like S,N,W, and E share the naming conventions with streets that specify the geographical position (i.e., N=North, S=South, W=West, E=East). This makes it difficult to programatically decide how a street like "N Groove St." should be converted into "North Grove Street" withut changing the "N Street" into "North Street". 
- For the zip code, I found that postal code 96816 does not corespond to Sacramento region, but rather to Honolulu. I removed the entries if that postal code.
- On a few occasions, I have cleaned the dataset for common misspellings. 

In [16]:
#all that is left is to load the json file in MongoDB

In [117]:
import pymongo
import pprint

def get_db():
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client.openStreetMaps
    return db

db=get_db()



print "Number of documents is "+str(db.sacramento.find().count())
print "Number of nodes is "+str(db.sacramento.find({"type":"node"}).count())
print "Number of way nodes is "+str(db.sacramento.find({"type":"way"}).count())
print "Number of unique users is "+str(len(db.sacramento.distinct("user")))

Number of documents is 1129233
Number of nodes is 1028822
Number of way nodes is 100346
Number of unique users is 982


In [120]:
#use list to access the result of the aggregation querry
#see http://stackoverflow.com/questions/30333020/mongodb-pymongo-aggregate-gives-strange-output-something-about-cursor
result=list(db.sacramento.aggregate([{"$group":{"_id":"$user","count":{"$sum":1}}},
                               {"$sort": {"count": -1}},{"$limit":5}] ))
pprint.pprint(result)

[{u'_id': u'T99', u'count': 203149},
 {u'_id': u'woodpeck_fixbot', u'count': 194983},
 {u'_id': u'jraller', u'count': 119323},
 {u'_id': u'nmixter', u'count': 88989},
 {u'_id': u'Eureka gold', u'count': 71309}]


In [125]:
#print most common 10 ammenities
result=list(db.sacramento.aggregate([
            {"$match":{"amenity":{"$exists":1}}}, #remove the ammenities with none fields
            {"$group":{"_id":"$amenity","count":{"$sum":1}}},
            {"$sort": {"count": -1}},
            {"$limit":10}
        ]))
pprint.pprint(result)

[{u'_id': u'parking', u'count': 1785},
 {u'_id': u'post_box', u'count': 1478},
 {u'_id': u'school', u'count': 814},
 {u'_id': u'place_of_worship', u'count': 605},
 {u'_id': u'restaurant', u'count': 409},
 {u'_id': u'fast_food', u'count': 299},
 {u'_id': u'fuel', u'count': 209},
 {u'_id': u'cafe', u'count': 151},
 {u'_id': u'bench', u'count': 139},
 {u'_id': u'toilets', u'count': 122}]


In [132]:
result=list(db.sacramento.aggregate([
            {"$match":{"amenity":{"$exists":1}, "amenity":"toilets"}},
            {"$group":{"_id":{"User":"$user"},"count":{"$sum":1}}},
            {"$sort": {"count": -1}},
            {"$limit":10}
        ]))

pprint.pprint(result)

[{u'_id': {u'User': u'T99'}, u'count': 19},
 {u'_id': {u'User': u'Charles_Smothers'}, u'count': 13},
 {u'_id': {u'User': u'BK_man'}, u'count': 11},
 {u'_id': {u'User': u'tjstansell'}, u'count': 10},
 {u'_id': {u'User': u'Bryce C Nesbitt'}, u'count': 10},
 {u'_id': {u'User': u'wallclimber21'}, u'count': 8},
 {u'_id': {u'User': u'Adam Mazurkiewicz'}, u'count': 6},
 {u'_id': {u'User': u'nmixter'}, u'count': 6},
 {u'_id': {u'User': u'animeigo'}, u'count': 4},
 {u'_id': {u'User': u'jraller'}, u'count': 4}]


In [135]:
#print top 5 denominations
result=list(db.sacramento.aggregate([
            {"$match":{"denomination":{"$exists":1}, "amenity":"place_of_worship"}}, #remove the denominations with none fields
            {"$group":{"_id":{"Denomination":"$denomination"},"count":{"$sum":1}}},
            {"$sort": {"count": -1}},
            {"$limit":5}
        ]))

pprint.pprint(result)

[{u'_id': {u'Denomination': u'baptist'}, u'count': 111},
 {u'_id': {u'Denomination': u'lutheran'}, u'count': 37},
 {u'_id': {u'Denomination': u'methodist'}, u'count': 35},
 {u'_id': {u'Denomination': u'catholic'}, u'count': 35},
 {u'_id': {u'Denomination': u'presbyterian'}, u'count': 21}]


In [136]:
#print top 5 religions
result=list(db.sacramento.aggregate([
            {"$match":{"religion":{"$exists":1}, "amenity":"place_of_worship"}}, #remove the religions with none fields
            {"$group":{"_id":{"Religion":"$religion"},"count":{"$sum":1}}},
            {"$sort": {"count": -1}},
            {"$limit":5}
        ]))

pprint.pprint(result)

[{u'_id': {u'Religion': u'christian'}, u'count': 560},
 {u'_id': {u'Religion': u'buddhist'}, u'count': 5},
 {u'_id': {u'Religion': u'muslim'}, u'count': 3},
 {u'_id': {u'Religion': u'unitarian_universalist'}, u'count': 2},
 {u'_id': {u'Religion': u'sikh'}, u'count': 2}]


In [123]:
#let's look at food stuff; Top 10 cuisines
result=list(db.sacramento.aggregate([
            {"$match":{"cuisine":{"$exists":1}}}, #remove the cuisines with none fields
            {"$group":{"_id":{"Food":"$cuisine"},"count":{"$sum":1}}},
            {"$sort": {"count": -1}},
            {"$limit":10}
        ]))

pprint.pprint(result)

[{u'_id': {u'Food': u'burger'}, u'count': 102},
 {u'_id': {u'Food': u'mexican'}, u'count': 75},
 {u'_id': {u'Food': u'coffee_shop'}, u'count': 61},
 {u'_id': {u'Food': u'sandwich'}, u'count': 49},
 {u'_id': {u'Food': u'pizza'}, u'count': 45},
 {u'_id': {u'Food': u'american'}, u'count': 21},
 {u'_id': {u'Food': u'chinese'}, u'count': 18},
 {u'_id': {u'Food': u'chicken'}, u'count': 17},
 {u'_id': {u'Food': u'italian'}, u'count': 16},
 {u'_id': {u'Food': u'japanese'}, u'count': 10}]


In [152]:
#find number of fast_food
print "Number of way fast foods is "+str(db.sacramento.find({"amenity":"fast_food"}).count())

Number of way nodes is 299


In [137]:
#look at fast food
result=list(db.sacramento.aggregate([
            {"$match":{"cuisine":{"$exists":1}, "amenity":"fast_food"}}, #remove the ammenities with none fields
            {"$group":{"_id":{"Food":"$cuisine"},"count":{"$sum":1}}},
            {"$sort": {"count": -1}},
            {"$limit":10}
        ]))

pprint.pprint(result)

[{u'_id': {u'Food': u'burger'}, u'count': 81},
 {u'_id': {u'Food': u'sandwich'}, u'count': 34},
 {u'_id': {u'Food': u'mexican'}, u'count': 33},
 {u'_id': {u'Food': u'pizza'}, u'count': 20},
 {u'_id': {u'Food': u'chicken'}, u'count': 14},
 {u'_id': {u'Food': u'chinese'}, u'count': 5},
 {u'_id': {u'Food': u'ice_cream'}, u'count': 3},
 {u'_id': {u'Food': u'japanese'}, u'count': 2},
 {u'_id': {u'Food': u'asian'}, u'count': 2},
 {u'_id': {u'Food': u'hotdog'}, u'count': 1}]


In [138]:
#names of popular fast food joints
#look at fast food
result=list(db.sacramento.aggregate([
            {"$match":{"name":{"$exists":1}, "amenity":"fast_food"}}, #remove the ammenities with none fields
            {"$group":{"_id":{"Name":"$name"},"count":{"$sum":1}}},
            {"$sort": {"count": -1}},
            {"$limit":10}
        ]))
pprint.pprint(result)

[{u'_id': {u'Name': u"McDonald's"}, u'count': 34},
 {u'_id': {u'Name': u'Taco Bell'}, u'count': 26},
 {u'_id': {u'Name': u'Subway'}, u'count': 22},
 {u'_id': {u'Name': u'Burger King'}, u'count': 15},
 {u'_id': {u'Name': u'KFC'}, u'count': 13},
 {u'_id': {u'Name': u'Jack in the Box'}, u'count': 9},
 {u'_id': {u'Name': u'A&W'}, u'count': 9},
 {u'_id': {u'Name': u"Carl's Jr."}, u'count': 8},
 {u'_id': {u'Name': u'Panda Express'}, u'count': 7},
 {u'_id': {u'Name': u'Del Taco'}, u'count': 7}]


In [111]:
#look at popular shops
result=list(db.sacramento.aggregate([
            {"$match":{"shop":{"$exists":1}}}, #remove the ammenities with none fields
            {"$group":{"_id":{"Shop":"$shop"},"count":{"$sum":1}}},
            {"$sort": {"count": -1}},
            {"$limit":10}
        ]))
pprint.pprint(result)

[{u'_id': {u'Shop': u'supermarket'}, u'count': 120},
 {u'_id': {u'Shop': u'car'}, u'count': 88},
 {u'_id': {u'Shop': u'clothes'}, u'count': 59},
 {u'_id': {u'Shop': u'convenience'}, u'count': 50},
 {u'_id': {u'Shop': u'department_store'}, u'count': 35},
 {u'_id': {u'Shop': u'hairdresser'}, u'count': 32},
 {u'_id': {u'Shop': u'shoes'}, u'count': 26},
 {u'_id': {u'Shop': u'beauty'}, u'count': 25},
 {u'_id': {u'Shop': u'car_repair'}, u'count': 23},
 {u'_id': {u'Shop': u'electronics'}, u'count': 22}]


In [145]:
#look at most popular shops
result=list(db.sacramento.aggregate([
            {"$match":{"shop":{"$exists":1}, "amenity":"shop"}}, #remove the ammenities with none fields
            {"$group":{"_id":{"Name":"$shop"},"count":{"$sum":1}}},
            {"$sort": {"count": -1}},
            {"$limit":10}
        ]))
pprint.pprint(result)

[{u'_id': {u'Name': u'clothes'}, u'count': 3},
 {u'_id': {u'Name': u'baby_goods'}, u'count': 1},
 {u'_id': {u'Name': u'beauty'}, u'count': 1},
 {u'_id': {u'Name': u'shoes'}, u'count': 1},
 {u'_id': {u'Name': u'car_repair'}, u'count': 1},
 {u'_id': {u'Name': u'interior_decoration'}, u'count': 1},
 {u'_id': {u'Name': u'alcohol'}, u'count': 1},
 {u'_id': {u'Name': u'doityourself'}, u'count': 1}]
